# Test cases requiring or benefiting from the context of a notebook

If the notebook runs successfully from start to finish, the test is successful!

TODO(all): Add additional tests and/or tests with particular assertions, as we encounter Python package version incompatibilities not currently detected by these tests.

In general, only add test cases here that require the context of a notebook. This is because this notebook, as currently written, will abort at the **first** failure. Compare this to a proper test suite where all cases are run, giving much more information about the full extent of any problems encountered.

# Package versions

In [ ]:
!pip3 freeze

In [ ]:
!pip3 show plotnine pandas google-cloud-storage google-cloud-bigquery google-resumable-media

# Test cases requiring the context of a notebook 

## Test Terra widgets

TODO(deflaux) this test could be enabled if we shared a Terra test workspace with the service account used for the GitHub Action.

# Test cases benefiting from the context of a notebook 

## Test AoU workbench snippets setup

We test these here since they don't test well via `pytest` because they all have the same module name.

In [ ]:
%%bash

git clone https://github.com/all-of-us/workbench-snippets.git /tmp/workbench-snippets

In [ ]:
%%bash

export WORKSPACE_CDR=placeholder_string
python3 /tmp/workbench-snippets/dataset-snippets/snippets_setup.py

In [ ]:
%%bash

export WORKSPACE_CDR=placeholder_string
python3 /tmp/workbench-snippets/sql-snippets/snippets_setup.py

In [ ]:
%%bash

export WORKSPACE_CDR=placeholder_string
python3 /tmp/workbench-snippets/storage-snippets/snippets_setup.py

## Test that our source control protections are working.

### global gitignore omit images and data from source control

Make some new files of various types in the git repo clone.

In [ ]:
%%bash

touch /tmp/workbench-snippets/should_be_ignored.png
touch /tmp/workbench-snippets/should_be_ignored.csv
touch /tmp/workbench-snippets/should_be_ignored.tsv
touch /tmp/workbench-snippets/should_be_ignored.CSV
touch /tmp/workbench-snippets/should_be_ignored.TSV
touch /tmp/workbench-snippets/should_be_visible.r
touch /tmp/workbench-snippets/should_be_visible.R
touch /tmp/workbench-snippets/should_be_visible.py
touch /tmp/workbench-snippets/should_be_visible.wdl
touch /tmp/workbench-snippets/should_be_visible.sh
touch /tmp/workbench-snippets/should_be_visible.md
mkdir -p /tmp/workbench-snippets/some_diretory/
touch /tmp/workbench-snippets/some_diretory/should_be_ignored.CSV
touch /tmp/workbench-snippets/some_diretory/should_be_visible.sh

The git status should only show code and documentation files (no data).

In [ ]:
actual = !cd /tmp/workbench-snippets/; git status

actual

In [ ]:
assert(any('should_be_visible' in s for s in actual))

In [ ]:
assert(not any('should_be_ignored' in s for s in actual))

### Ensure that notebook outputs are removed

Make a notebook file containing outputs.

In [ ]:
%%bash

cat << 'EOF' > /tmp/workbench-snippets/should_be_visible.ipynb

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "pacific-album",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'this is an output'"
      ]
     },
     "execution_count": 2,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "\" \".join(['this', 'is', 'an', 'output'])"
   ]
  }
 ],
 "metadata": {},
 "nbformat": 4,
 "nbformat_minor": 5
}

EOF

The git diff should show that the outputs have been removed.

In [ ]:
actual = !cd /tmp/workbench-snippets/ ; git add should_be_visible.ipynb ; git diff --staged should_be_visible.ipynb

actual

In [ ]:
assert(any('"outputs": [],' in s for s in actual))

In [ ]:
assert(not any('this is an output' in s for s in actual))